<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Ciencia_de_Dados/blob/main/Spark_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master('local[*]')\
    .appName("Regressao com Spark")\
    .getOrCreate()

spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dados = spark.read.json('/content/drive/MyDrive/Arquivos/imoveis.json')

In [ ]:
dados

In [ ]:
dados.show(truncate=False)

In [ ]:
dados.count()

In [ ]:
dados.printSchema()

In [ ]:
from os import truncate
dados\
    .select('ident.customerID', 'listing.*') \
    .show(truncate=False)

In [ ]:
dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .show(truncate=False)

In [ ]:
dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .drop("city", "location", "totalAreas")\
    .show(truncate=False)


In [ ]:
dataset = dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .drop("city", "location", "totalAreas")\
    #.show(truncate=False)


In [ ]:
dataset.printSchema()


In [ ]:
from pyspark.sql.types import IntegerType, DoubleType

In [ ]:
dataset\
    .withColumn('usableAreas', dataset['usableAreas'].cast(IntegerType()))\
    .withColumn('price', dataset['price'].cast(DoubleType()))\
    .withColumn('condo', dataset['condo'].cast(DoubleType()))\
    .withColumn('iptu', dataset['iptu'].cast(DoubleType()))\
    .printSchema()

In [ ]:
dataset = dataset\
    .withColumn('usableAreas', dataset['usableAreas'].cast(IntegerType()))\
    .withColumn('price', dataset['price'].cast(DoubleType()))\
    .withColumn('condo', dataset['condo'].cast(DoubleType()))\
    .withColumn('iptu', dataset['iptu'].cast(DoubleType()))

In [ ]:
dataset.show()

In [ ]:
dataset\
    .select('usage')\
    .groupBy('usage')\
    .count()\
    .show()

In [ ]:
dataset\
    .select('*')\
    .where('usage=="Residencial"')

In [ ]:
dataset\
    .select('unit')\
    .groupBy('unit')\
    .count()\
    .show()

In [ ]:
dataset\
    .select('zone')\
    .groupBy('zone')\
    .count()\
    .show()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
dataset\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
    .show()

In [ ]:
dataset\
    .select('*')\
    .na\
    .fill(0)\
    .show()

In [ ]:
dataset = dataset\
    .select('*')\
    .na\
    .fill(0)

In [ ]:
dataset\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
    .show()

In [ ]:
dataset\
    .select('zone')\
    .groupBy('zone')\
    .count()\
    .show()


In [ ]:
dataset = dataset\
    .where(f.col('zone') != '')

In [ ]:
dataset.show()

###Preparando os Dados


##Variáveis Dummy

In [ ]:
dataset\
    .groupBy('customerID')\
    .pivot('unit')\
    .agg(f.lit(1))\
    .na\
    .fill(0)\
    .show()

In [ ]:
unit = dataset\
    .groupBy('customerID')\
    .pivot('unit')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

In [ ]:
dataset\
    .groupBy('customerID')\
    .pivot('zone')\
    .agg(f.lit(1))\
    .na\
    .fill(0)\
    .show()

In [ ]:
zone = dataset\
    .groupBy('customerID')\
    .pivot('zone')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

In [ ]:
dataset = dataset\
    .join(unit, 'customerID', how='inner')\
    .join(zone, 'customerID', how='inner')

In [ ]:
dataset.show()

##Vetoriacao

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset = dataset.withColumnRenamed('price','label')

In [ ]:
X = [
    'bathrooms',
    'bedrooms',
    'floors',
    'parkingSpaces',
    'suites',
    'unitFloor',
    'unitsOnTheFloor',
    'usableAreas',
    'condo',
    'iptu',
    'Apartamento',
    'Casa',
    'Outros',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul'
]

In [ ]:
assembler = VectorAssembler(inputCols=X, outputCol= 'features')
assembler

In [ ]:
dataset.show()

In [ ]:
dataset_prep = assembler.transform(dataset).select('features', 'label')

In [ ]:
dataset_prep.show(10, truncate=False)

In [ ]:
from pyspark.ml.stat import Correlation
import pandas as pd


In [ ]:
correlacao = Correlation.corr(dataset_prep, 'features').collect()

In [ ]:
correlacao

In [ ]:
correlacao = Correlation.corr(dataset_prep, 'features').collect()[0][0]

In [ ]:
correlacao

In [ ]:
correlacao.toArray()

In [ ]:
dataframe_correlacao = pd.DataFrame(correlacao.toArray(), columns=X, index=X)

In [ ]:
dataframe_correlacao

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,10))
paleta = sns.color_palette("light:salmon", as_cmap=True)
sns.heatmap(dataframe_correlacao.round(1), annot=True, cmap=paleta)